In [7]:
pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install langchain

In [10]:
!pip install openai

In [11]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

Note: you may need to restart the kernel to use updated packages.


In [12]:
!pip install -qU langchain-openai

In [13]:
!pip3 install faiss-cpu

  Using cached faiss_cpu-1.8.0-cp39-cp39-win_amd64.whl (14.5 MB)


In [4]:
import getpass
import os

openai_api_key = os.environ.get("OPENAI_API_KEY")
#GPT API 키 설정

In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
langchain_api_key = os.environ.get("LANGCHAIN_API_KEY")
#Langchain API 키 설정

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature = 0)
#3.5turbo모델, temperature가 0이면 같은 질문에 같은 대답

In [20]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [21]:
url = "https://n.news.naver.com/article/437/0000378416"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)
docs = loader.load()
#웹페이지의 Data를 이용하여 RAG

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(docs)
#뉴스 기사를 Split

In [23]:
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
#FAISS VectorStore를 생성

In [24]:
retriever = vectorstore.as_retriever()
#뉴스에 포함되어 있는 정보를 검색하고 생성

In [25]:
prompt = hub.pull("rlm/rag-prompt")
#프롬포트 생성

In [26]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
# 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.

In [27]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
#Chain 생성

In [28]:
question = "부영그룹의 출산 장려 정책에 대해 설명해주세요"
response = rag_chain.invoke(question)
#질문과 답변을 받기

In [29]:
print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

URL: https://n.news.naver.com/article/437/0000378416
문서의 수: 1
[HUMAN]
부영그룹의 출산 장려 정책에 대해 설명해주세요

[AI]
부영그룹은 출산 장려 정책으로 2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 지속할 계획이며, 연년생과 쌍둥이 자녀가 있는 경우 2억원을 받을 수 있습니다. 또한, 셋째까지 낳는 경우 국민주택을 제공할 예정이며, 출산장려금을 받는 직원들의 세금 부담을 고려해 정부가 면세해달라는 제안도 나왔습니다. 해당 정책은 점점 확산되는 분위기 속에서 기업들도 출산 장려책을 실시하고 있습니다.
